In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable04.csv")
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,27]
df = df.drop(df.columns[cols],axis=1)
df["phospho_site"] =  df["geneName"] + "_" + df["variableSites"] 

df

,variableSites,geneName,AO-A12D.01TCGA,C8-A131.01TCGA,AO-A12B.01TCGA,E2-A10A.02TCGA,BH-A18Q.02TCGA,C8-A130.02TCGA,C8-A138.03TCGA,E2-A154.03TCGA,...,A2-A0SW.35TCGA,AO-A0JL.35TCGA,BH-A0BV.35TCGA,A2-A0YM.36TCGA,BH-A0C7.36TCGA,A2-A0SX.36TCGA,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC,phospho_site
0,S1007s,AHNAK,-0.919089,-2.565861,0.177221,0.203602,-0.086607,-2.700983,1.301494,2.952304,...,4.389396,1.338505,3.450978,0.200324,1.728556,1.092898,-1.153228,0.234135,2.595100,AHNAK_S1007s
1,S1010s,AHNAK,-1.271514,-1.726458,-0.320742,-0.523359,-0.315189,-2.208631,1.820337,3.049420,...,0.070029,0.495032,0.776968,1.017243,2.213385,0.444430,-0.331621,0.347756,2.527968,AHNAK_S1010s
2,S1023s,AHNAK,NaN,NaN,NaN,NaN,NaN,NaN,-0.302638,3.157830,...,0.812259,0.364182,1.626879,1.985442,-0.342381,1.767384,NaN,NaN,NaN,AHNAK_S1023s
3,S1042s,AHNAK,-2.298519,-0.370305,-1.169382,0.330244,0.819633,-0.419166,1.570435,0.883498,...,0.797827,1.628386,1.382329,NaN,NaN,NaN,NaN,NaN,NaN,AHNAK_S1042s
4,S1068s,AHNAK,-0.802335,-2.023904,-1.223738,-0.465794,-1.585542,-1.411369,1.394314,2.412517,...,2.065803,3.490470,2.797509,0.485597,2.001793,-0.352144,NaN,NaN,NaN,AHNAK_S1068s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62674,T210t,GPR78,0.014945,2.483162,-0.324249,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GPR78_T210t
62675,S217s T218t T224t,SLC27A6,-0.698553,-1.693688,-0.175210,0.729908,0.613301,-0.054275,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SLC27A6_S217s T218t T224t
62676,T47t,C19ORF48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C19ORF48_T47t
62677,S250s,PIGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PIGL_S250s


In [3]:

df1 = df.set_index("phospho_site", "sample_ID")

df1 = df1.drop(["variableSites","geneName"], axis = 1)


In [4]:
df2 = df1.T
df2

phospho_site,AHNAK_S1007s,AHNAK_S1010s,AHNAK_S1023s,AHNAK_S1042s,AHNAK_S1068s,AHNAK_S106s,AHNAK_S106s S109s,AHNAK_S1088s,AHNAK_S110s,AHNAK_S1123s,...,BST1_S40s,WNT9B_T234t,C4ORF39_S92s T107t,AKD1_T1425t,ZNF329_T359t,GPR78_T210t,SLC27A6_S217s T218t T224t,C19ORF48_T47t,PIGL_S250s,RS1_Y165y Y166y T170t
AO-A12D.01TCGA,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,NaN,NaN,NaN,-7.085443,-3.870377,0.014945,-0.698553,NaN,NaN,NaN
C8-A131.01TCGA,-2.565861,-1.726458,NaN,-0.370305,-2.023904,0.637987,0.587573,0.693443,1.003493,-0.403074,...,NaN,NaN,NaN,1.676528,-3.110339,2.483162,-1.693688,NaN,NaN,NaN
AO-A12B.01TCGA,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,NaN,NaN,NaN,-0.213785,-2.580861,-0.324249,-0.175210,NaN,NaN,NaN
E2-A10A.02TCGA,0.203602,-0.523359,NaN,0.330244,-0.465794,0.540767,NaN,-4.867033,0.386164,0.754579,...,NaN,NaN,NaN,NaN,-0.449347,NaN,0.729908,NaN,NaN,NaN
BH-A18Q.02TCGA,-0.086607,-0.315189,NaN,0.819633,-1.585542,-0.909909,NaN,-1.963816,1.058329,2.100100,...,NaN,NaN,NaN,NaN,-2.060913,NaN,0.613301,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BH-A0C7.36TCGA,1.728556,2.213385,-0.342381,NaN,2.001793,1.182082,0.239081,NaN,0.925506,1.260388,...,NaN,-0.129123,NaN,NaN,-2.434978,NaN,NaN,NaN,NaN,NaN
A2-A0SX.36TCGA,1.092898,0.444430,1.767384,NaN,-0.352144,0.488462,1.204979,NaN,1.345080,0.780673,...,NaN,-2.493687,NaN,NaN,-2.523709,NaN,NaN,NaN,NaN,NaN
263d3f-I.CPTAC,-1.153228,-0.331621,NaN,NaN,NaN,NaN,NaN,-0.603446,0.103708,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
blcdb9-I.CPTAC,0.234135,0.347756,NaN,NaN,NaN,NaN,NaN,0.358681,0.898382,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
mutations = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable01.csv")
mutations = mutations.set_index("Sample ID")
mutations = mutations[["PIK3CA Mutation"]]
mutations = mutations.rename(columns = {"PIK3CA Mutation" : "PIK3CA_Mutation"})
mutations =  mutations.replace(np.nan, 'Wildtype', regex=True)
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense_Mutation|Somatic|p.H1047R
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense_Mutation|Somatic|p.E545K
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [30]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E542K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E545K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('H1047R')] = "Hotspot"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Hotspot
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Hotspot
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [33]:

wt_hotspot_df = mutations[mutations.PIK3CA_Mutation.str.contains('Hotspot') | 
                    mutations.PIK3CA_Mutation.str.contains('Wildtype')]

In [34]:
wt_hotspot_df

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Hotspot
A2-A0EX,Hotspot
A2-A0SW,Wildtype
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype
